In [1]:
# filite 태그 검색후 게시물의 url을 얻어와서 각각의 크롤링을 진행함
from bs4 import BeautifulSoup
import selenium.webdriver as webdriver
import urllib.parse
from urllib.request import Request, urlopen
from time import sleep
import pandas as pd
import datetime


import pymysql 
from sqlalchemy import create_engine
import time
pymysql.install_as_MySQLdb()
import MySQLdb
from multiprocessing import Pool

import pickle

from konlpy.tag import Okt
from konlpy.tag import Kkma
from konlpy.utils import pprint

import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
from wordcloud import WordCloud
import math
import numpy as np

In [2]:
class insta_analysis :
    
    def __init__(self):
        print("Insta analysis")
        
    ## ----Instagram Crawling function ine----
    
    def search_tag(self,tag,chrome_driver_root):
        
        
        chrome_driver = chrome_driver_root
        driver = webdriver.Chrome(executable_path=chrome_driver)

        search = tag
        search = urllib.parse.quote(search)
        url = 'https://www.instagram.com/explore/tags/'+str(search)+'/'
        driver.get(url) 
        sleep(5)


        SCROLL_PAUSE_TIME = 1.0

        realtitle = []
        img_type = []
        upload_date = []
        reallink = []
        date_obj = None
        while True:
            pageString = driver.page_source
            bsObj = BeautifulSoup(pageString, "lxml")
            sleep(SCROLL_PAUSE_TIME)
            for link1 in bsObj.find_all(name="div",attrs={"class":"Nnq7C weEfm"}):
                
                for idx in range(len(link1)):
                    try:
                        title = link1.select('a')[idx] 
                        real = title.attrs['href']
                        reallink.append(real) 

                        temp_title = str(link1.select("img")[idx]).split('''class=''')[0].split("img alt=")[-1].strip().strip('''"''')
                        if "이미지: " in temp_title :
                            title_type = temp_title.split("이미지: ")[-1]

                            if " on " in temp_title :
                                date_str = temp_title.split("이미지: ")[-2].split(" on ")[-1].strip(". ")

                                if " tagging " in date_str:
                                    date_str = date_str.split(" tagging ")[0]
                                date_obj = datetime.datetime.strptime(date_str,'%B %d, %Y').date()
                            else :
                                pass # 앞 뒤 데이터의 날짜가 동일하다는 가정하 데이터 누락시 전 데이터 날짜 입력
                            real_title = temp_title.split("이미지: ")[-2].split(" on ")[0].strip("Photo by ")
                        else :
                            title_type = None
                            real_title = temp_title

                        img_type.append(title_type)
                        upload_date.append(date_obj)
                        realtitle.append(real_title)
                    except :
                        print("err happened")
              
                    
            last_height = driver.execute_script("return document.body.scrollHeight")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                sleep(SCROLL_PAUSE_TIME*5)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break

                else:
                    last_height = new_height
                    continue

        reallinknum = len(reallink)
        print(str(reallinknum)+"uploaded.")
        
        # In case of crawling instagram, the insta corp restrict crawling speed 
        # so your return value is not equal to searching result in Instagram by yourself 

        insta_search_df = pd.DataFrame({"date":upload_date, "title":realtitle, "img_type":img_type, "link":reallink})
        driver.close()
        
        return insta_search_df
    
    def hashtag(self,linklist):
        SCROLL_PAUSE_TIME = 1.0
        hash_tag_df = pd.DataFrame(columns = ['link','hashtag'])
        err_list = []
        for link in linklist:
            
            try:
                url = "https://www.instagram.com/p"+link

                req = Request(url,headers = {'User-Agent':'Mozilla/5.0'})
                src = urlopen(req).read()

                soup = BeautifulSoup(src,"lxml",from_encoding = 'utf-8')
                soup1 = soup.find("meta",attrs={"property":"og:description"})
                reallink1 = soup1['content']
                reallink1 = reallink1[reallink1.find("@")+1:reallink1.find(")")]
                reallink1 = reallink1[:]

                if reallink1 == '':
                    reallink1 = 'Null'

                for reallink2 in soup.find_all("meta",attrs={"property":"instapp:hashtags"}): 
                    reallink2 = reallink2['content']
                    hash_tag_df = hash_tag_df.append({"link":link,"hashtag":reallink2},ignore_index=True )
            except:
                err_list.append(link)
                sleep(SCROLL_PAUSE_TIME*5)
                print("err happened")
            print(link + 'completed')
        return hash_tag_df, err_list

    ##--------- for Mysql ----------
    def Mysql_df_save(self,df_save,id_,pw_,host,DB,table_name):

        engine = create_engine("mysql+mysqldb://{}:".format(id_)+"{}".format(pw_)+"@{}/{}".format(host,DB), encoding='utf-8')
        conn = engine.connect()
        df_save.to_sql(name='{}'.format(table_name), con=engine, if_exists='append',index=False)

        print("=====Saving dataframe to Mysql is completed=====")

    def Mysql_df_load(self,id_,pw_,host,port,DB,table_name):

        db = pymysql.connect(host = '{}'.format(host), port = port,user ='{}'.format(id_),passwd = '{}'.format(pw_),db='{}'.format(DB),charset = 'utf8')
        cursor = db.cursor()
        sql = """ select * from {} ;
                  """.format(table_name)
        cursor.execute(sql)
        data = pd.DataFrame(cursor.fetchall())

        return data
    
    
    ##---------- data analysis ------------
    def classifying_language(self, dataframe):
    
        # output list
        Kor_url_list = []
        Eng_url_list = []
        Etc_url_list = []

        for index in dataframe.index:
            if isHangul(dataframe.loc[index].title):
                Kor_url_list.append(dataframe.loc[index].url)
            elif isEnglish(dataframe.loc[index].title):
                Eng_url_list.append(dataframe.loc[index].url)
            else : 
                text_list = dataframe.loc[index].title.split()
                
                # Sometimes, distribution error happened
                # this err is empirical detection
                wrong_count = 0
                correct_count = 0

                for idx in text_list :
                    if 'ko' == langid.classify(idx)[0] :
                        correct_count += 1
                    else :
                        wrong_count+= 1

                if correct_count > wrong_count :
                    Kor_url_list.append(dataframe.loc[index].url)

                else :     
                    Etc_url_list.append(dataframe.loc[index].url)

        return Kor_url_list, Eng_url_list, Etc_url_list
    
    ##  ----restrict timeline----
    
    def select_date(self,df_input,str_start,str_end):
    
        # type str date input change to type datetime
        # column name need to comment_date
        
        dt_start = datetime.datetime.strptime(str_start,"%Y-%m-%d").date()
        dt_end = datetime.datetime.strptime(str_end,"%Y-%m-%d").date()
        start_to_df_input = df_input[df_input["comment_date"] >= dt_start]
        start_to_end_df_input = start_to_df_input[start_to_df_input["comment_date"] <= dt_end]

        return start_to_end_df_input
    
    ##  ----distribute morpheme----
    
    def tokenizer_lang_class(self, df):
    
        print("-------------- tokenizing language start ----------------")    
        okt = Okt()
        res_token_df = pd.DataFrame(columns = ['token','type'])
        df.reset_index(drop = True,inplace=True)
        k = 0     

        for idx in df.index :
            # step 출력
            if idx % 100 == 0 :
                print("step = ",idx, df.loc[idx])

            # tokenize
            token_list = okt.pos(df.loc[idx].hashtag) 

            # 새로운 데이터 프레임에 적용
            for token in token_list :    
                if token[-1] in ['Noun','Verb','Adjective'] :
                    res_token_df.loc[k] = [token[0],token[-1]]
                    k += 1

        N_token = res_token_df[res_token_df["type"] == 'Noun']
        V_token = res_token_df[res_token_df["type"] == 'Verb']
        A_token = res_token_df[res_token_df["type"] == 'Adjective']         

        return N_token, V_token, A_token
    
    def morpheme_distribution(self,df):

        print("-------------- morpheme_distribution start ----------------")
        kkma = Kkma()
        NN_token_df = pd.DataFrame(columns = ['token','type'])
        VV_token_df = pd.DataFrame(columns = ['token','type'])
        VA_token_df = pd.DataFrame(columns = ['token','type'])
        etc_token_df = pd.DataFrame(columns = ['token','type'])

        df.reset_index(drop = True,inplace=True)
        k = 0     

        for idx in df.index :
            # step 출력
            if idx % 100 == 0 :
                print("step = ",idx, df.loc[idx])

            # tokenize
            token_list = kkma.pos(df.loc[idx].token) 

            temp_token = ''
            for token in token_list :
                if 'N' in token[-1]: 
                    temp_token += token[0]     
            if temp_token == df.loc[idx].token :
                NN_token_df.loc[k] = [df.loc[idx].token,token_list[0][-1]]
                k += 1
            else :
                for token in token_list :
                    if 'VV' == token[-1]:
                        VV_token_df.loc[k] = [token[0],token[-1]]
                    elif 'VA' == token[-1]: 
                        VA_token_df.loc[k] = [token[0],token[-1]]
                    elif 'NN' in token[-1]: 
                        NN_token_df.loc[k] = [token[0],token[-1]]
                    else :
                        etc_token_df.loc[k] = [token[0],token[-1]]
                k += 1
        NN_token_df.reset_index(drop = True,inplace=True)
        VV_token_df.reset_index(drop = True,inplace=True)
        VA_token_df.reset_index(drop = True,inplace=True)
        etc_token_df.reset_index(drop = True,inplace=True)

        return NN_token_df, VV_token_df, VA_token_df, etc_token_df
    
    def save_pickle(self, df,name):
        df.to_pickle("{}".format(name))
        print("{} save is completed".format(name))
    
    ##  ----making word cloud----

    
    def make_wordcloud(self,list_,count,save_name):

        counts = Counter(list_) 

        words = dict(counts.most_common(count))
        
        try :
            words.pop(save_name)
        except :
            pass
        try :
            words.pop('?')
        except :
            pass
        try :
            words.pop('맞팔')
        except :
            pass
        try :     
            words.pop('선팔')
        except :
            pass
        try :
            words.pop('선팔하면맞팔')
        except :
            pass
        try :
            words.pop('좋아요')
        except :
            pass
        try :
            words.pop('소통')
        except :
            pass 
        
        try :
            words.pop('좋반')
        except :
            pass
        
        try :
            words.pop('??')
        except :
            pass
    
            
            
        font_path = 'c:\\windows\\fonts\\NanumGothic.ttf'
        wc = WordCloud(font_path= font_path ,
                       background_color="white",
                       colormap = "Accent_r",
                       width = 1500,
                       height = 1000
                       ).generate_from_frequencies(words)

        plt.imshow(wc)
        plt.axis('off')
        plt.show()
        plt.imshow(wc)
        plt.savefig('wordcloud_{}'.format(save_name),dpi = 300)


In [6]:
if __name__ == '__main__':
    start_time = time.time()
    chrome_driver_root = r'C:/Users/user/Desktop/chromedriver_win32/chromedriver.exe'
    insta = insta_analysis()
    db = pymysql.connect(host = 'localhost', port = 3306,user ='root',passwd = 'sogangsp',db='sns_db',charset = 'utf8')
    cursor = db.cursor()
    sql1 = """alter table insta_tag_asahi add column id int(9) not null auto_increment primary key first;"""
    cursor.execute(sql1)
    sql2 = """delete from insta_tag_asahi
              where id
              not in (select * from (select MIN(id) from insta_tag_asahi group by title, link) as temptable);"""
    cursor.execute(sql2)
    
    df2 = insta.Mysql_df_load(id_ = 'root'
                              ,pw_ = 'sogangsp'
                              ,host = 'localhost'
                              ,port =3306
                              ,DB = 'sns_db'
                              ,table_name = 'insta_tag_asahi')
    df2.rename({1:"date", 2:"title", 3:"img_type", 4:"link"},axis = 1,inplace = True)
    hashtag_asahi,err2 = insta.hashtag(df2.link)
    insta.Mysql_df_save(hashtag_asahi
                  ,id_ = 'root'
                  ,pw_ = 'sogangsp'
                  ,host = 'localhost'
                  ,DB = 'sns_db'
                  ,table_name = 'hashtag_asahi')
    print("---- %s seconds ----" % (time.time() - start_time))
    
    ## 중복데이터가 발생
    ## insta 자체에서 중복데이터를 지속적으로 보내줌
"""
Mysql 실행

## 고유 id auto-increment 생성
1. alter table [table_name] add column id int(9) not null auto_increment primary key first;

## 중복컬럼 존재하는 
## hashtag table : hashtag, link / insta tag table : title, link 중복컬럼 그룹화를 통해 중복값 1개 남기고 제거 
2. delete from [table_name]
   where id
   not in (select * from (select MIN(id) from [table_name] group by [column1, column2]) as temptable);

"""

Insta analysis


C:\Users\user\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1681, 'Integer display width is deprecated and will be removed in a future release.')
  result = self._query(query)


/p/B01u-Mzget7/completed
/p/B5ECnnqHRq2/completed
/p/B5G2_yNnMq5/completed
/p/B5EJ4rbF3mY/completed
/p/Bxe-9RHAOZf/completed
/p/B-vYNRyH2F2/completed
/p/B5EMNQSF9H1/completed
/p/BtFc9zHAd2J/completed
/p/BwT7YIbHqMZ/completed
/p/B_Pj9ieFYyY/completed
/p/B_Pg5u0DEQP/completed
/p/B_PSCqRFDf5/completed
/p/B_LIWhQheGl/completed
/p/B_H2XSHney_/completed
/p/B_Hw-UjFnvn/completed
/p/B_HHy2lpqJu/completed
/p/B_FeMHopj1S/completed
/p/B_FSGf6jKGz/completed
/p/B_CqjdVHDdk/completed
/p/B-9Lg3oj-Wu/completed
/p/B-7YL3cjx-L/completed
/p/B-57ayxjHKe/completed
/p/B-37v4PDhb1/completed
/p/B-2IJeTHp1m/completed
/p/B-1PJE3FslZ/completed
err happened
/p/B-0f1U0lZHc/completed
/p/B-zT5penx8C/completed
/p/B-sHmC2BpEf/completed
/p/B-rw1_oJy4E/completed
/p/B-q2rj3hd5w/completed
/p/B-llsnqnLSd/completed
/p/B-ljfu1nUuZ/completed
/p/B-lX3tYBRhC/completed
/p/B01u-Mzget7/completed
/p/B5ECnnqHRq2/completed
/p/B5G2_yNnMq5/completed
/p/B5EJ4rbF3mY/completed
/p/Bxe-9RHAOZf/completed
/p/B-vYNRyH2F2/completed
/p/B5EMNQSF9

/p/B9LF5nyF-VX/completed
/p/B9J6iUPlaqV/completed
/p/B9J5f9wFuW4/completed
/p/B9JlVWMnO9s/completed
/p/B9HZbPXnR5-/completed
/p/B9HYIcrhoRi/completed
/p/B01u-Mzget7/completed
/p/B5ECnnqHRq2/completed
/p/B5G2_yNnMq5/completed
/p/B5EJ4rbF3mY/completed
/p/Bxe-9RHAOZf/completed
/p/B-vYNRyH2F2/completed
/p/B5EMNQSF9H1/completed
/p/BtFc9zHAd2J/completed
/p/BwT7YIbHqMZ/completed
/p/B9gAQL2JkKa/completed
/p/B8tpiOYBSC2/completed
/p/B4B1jbMp59-/completed
/p/Bq1W0pHl3G8/completed
/p/BoB8cm-nTNM/completed
/p/B9f9wjvgrct/completed
/p/B9ew3W5pCpw/completed
/p/B9eRJUapYqG/completed
/p/B9eIXH1B07N/completed
/p/B9as42YgAUl/completed
/p/B9YbSpnn-N0/completed
/p/B9WokxanHma/completed
/p/B9Ty1duJRUn/completed
/p/B9RZ-bZHXnB/completed
/p/B9Q_AcygTRD/completed
/p/B9Q1_tQH2U0/completed
/p/B9PkkcDlO9G/completed
/p/B9O1TVZp6f9/completed
/p/B9OV11YHiw7/completed
/p/B9N5RD5Bdxh/completed
/p/B9MNu-jJzMR/completed
/p/B9LvQNYh6V1/completed
/p/B9LW451FIop/completed
/p/B9LQVLZFihb/completed
/p/B9LF5nyF-VX/completed


/p/B8JaHizJNag/completed
/p/B8GyR3OnB7G/completed
/p/B8EfKpIjvfh/completed
/p/B8ETcXejhSH/completed
/p/B8ELwF7hVWq/completed
/p/B8D4icLH5F7/completed
/p/B8C7E60Ad8A/completed
/p/B8BWWY1AyjX/completed
/p/B8AauIMnwf7/completed
/p/B78kpoulmD0/completed
/p/B77vF-FFizm/completed
/p/B764kiGHTdP/completed
/p/B75-UdlBrer/completed
/p/B755cb-Bfhn/completed
/p/B75Qy69p_ZP/completed
/p/B73HUzll9eB/completed
/p/B70cuQTFSRa/completed
/p/B7zkq5Lpami/completed
/p/B7ykvn4gPoB/completed
/p/B7xnfP-AP8c/completed
/p/B7v9TP_A0S6/completed
/p/B7uv6Lyl-Oy/completed
/p/B7uKmN8hK1a/completed
/p/B7r7EpYnChU/completed
/p/B7qbdICgWwq/completed
/p/B7oAY6wFpwM/completed
/p/B7luu9yJv-H/completed
/p/B7lWqXdgikN/completed
/p/B7i-dTYgVj3/completed
/p/B7iu3LIHKNW/completed
/p/B7cGlZaFZbz/completed
/p/B7atfvdFrQx/completed
/p/B7U_UIMhFnf/completed
/p/B01u-Mzget7/completed
/p/B5ECnnqHRq2/completed
/p/B5G2_yNnMq5/completed
/p/B5EJ4rbF3mY/completed
/p/Bxe-9RHAOZf/completed
/p/B-vYNRyH2F2/completed
/p/B5EMNQSF9H1/completed


...

/p/BjG6-gbF4Us/completed
/p/BjGsrzNlciQ/completed
/p/BjFeIg7hCWW/completed
/p/BjFSgrjhhUb/completed
/p/BjFQ7rZg2wg/completed
/p/BjFIpwOF2jj/completed
/p/BjEn_l3Fpeg/completed
/p/BjEmpXXAD2Z/completed
/p/BjEZwRKFYyP/completed
/p/BjEI__CF2rz/completed
/p/BjEHpbPlgqj/completed
/p/BjD4_khlgkD/completed
/p/BjD3oCBAQjg/completed
/p/BjDkHTYgg-9/completed
/p/BjDTscrHMZK/completed
/p/BjCeIs8nKA3/completed
/p/BjCcbNvH87c/completed
/p/BjCbdTOBBDM/completed
/p/BjCa5T9Ha6K/completed
/p/BjCK5f3AS8g/completed
/p/BjB9sjlgM63/completed
/p/BjB8dXslWPY/completed
/p/BjB5Gg2jLAO/completed
/p/B01u-Mzget7/completed
/p/B5ECnnqHRq2/completed
/p/B5G2_yNnMq5/completed
/p/B5EJ4rbF3mY/completed
/p/Bxe-9RHAOZf/completed
/p/B-vYNRyH2F2/completed
/p/B5EMNQSF9H1/completed
/p/BtFc9zHAd2J/completed
/p/BwT7YIbHqMZ/completed
/p/BjHja6JlrB5/completed
/p/BjHid_jg-h-/completed
/p/BjHNrAUjRFd/completed
/p/BjHLjYJgiYY/completed
/p/BjHA9F8AIXK/completed
/p/BjG_BbnndDS/completed
/p/BjG8-ueAbkl/completed
/p/BjG6-gbF4Us/completed


/p/B5G2_yNnMq5/completed
/p/B5EJ4rbF3mY/completed
/p/Bxe-9RHAOZf/completed
/p/B-vYNRyH2F2/completed
/p/B5EMNQSF9H1/completed
/p/BtFc9zHAd2J/completed
/p/BwT7YIbHqMZ/completed
/p/Bi7B1zwDYSK/completed
/p/Bi613FaAl1W/completed
/p/Bi6ytEBnajc/completed
/p/Bi6xsMQggGR/completed
/p/Bi6lmPEA3Br/completed
/p/Bi6VqCJDZ1f/completed
/p/Bi41owjF_Wk/completed
/p/Bi4uOzfFR9i/completed
/p/Bi4i_oPAWnd/completed
/p/Bi4gtF9gBWi/completed
/p/Bi4c1lmAmu8/completed
/p/Bi4Zbs-BSOG/completed
/p/Bi4RmOdlhHm/completed
/p/Bi4Rcu7BVq6/completed
/p/Bi4QXZmB_EG/completed
/p/Bi4QV6clBc-/completed
/p/Bi4NNF_gQYp/completed
/p/Bi4H7BRBNIe/completed
/p/Bi4Hmk3A2pa/completed
/p/Bi4F8JNBnu5/completed
/p/Bi3xUOED4AC/completed
/p/Bi3pJg2AztG/completed
/p/Bi3dfvCAdV8/completed
/p/Bi3cnTOgTjG/completed
/p/Bi3WRMYnZey/completed
/p/Bi3LQyBg1P0/completed
/p/Bi26tlqnVAR/completed
/p/Bi22yMzBIZz/completed
/p/Bi2oLiSHzR6/completed
/p/Bi2Kd_Hn8lN/completed
/p/Bi18ulogk3d/completed
/p/Bi2Jut0HgCa/completed
/p/Bi2AbZNB6ZB/completed


/p/Bivt6kRgJ9_/completed
/p/Bivtfp6H0--/completed
/p/BivtdKQA1DP/completed
/p/BivnxWfghjH/completed
/p/BivkHsfAirx/completed
/p/BivQEVPhm9K/completed
/p/BiucQFoFVKr/completed
/p/BiuXH6XFmVY/completed
/p/BiuU1K5Ahxd/completed
/p/BiuTpvJFs7G/completed
/p/BiuTHxGnsYs/completed
/p/BiuPS9DHz8P/completed
/p/BiuNTkQAu_r/completed
/p/BiuNCFSn4uA/completed
/p/BiuKwSknxY1/completed
/p/BiuHdpXn1Ys/completed
/p/BiuHQQiAuGq/completed
/p/BiuFdCZgBQ8/completed
/p/BiuFaCIlOgy/completed
/p/BiuEoCwBspi/completed
/p/Bit-8QeBfaR/completed
/p/B01u-Mzget7/completed
/p/B5ECnnqHRq2/completed
/p/B5G2_yNnMq5/completed
/p/B5EJ4rbF3mY/completed
/p/Bxe-9RHAOZf/completed
/p/B-vYNRyH2F2/completed
/p/B5EMNQSF9H1/completed
/p/BtFc9zHAd2J/completed
/p/BwT7YIbHqMZ/completed
/p/BiweQvPlkl5/completed
/p/BiwZDA6FMdB/completed
/p/BiwSUoXgKap/completed
/p/BiwPjF5AtsR/completed
/p/BiwH8LylfLM/completed
/p/BiwBHzFni7k/completed
/p/Biv1e77HDy4/completed
/p/Bivyk5TFCRs/completed
/p/Bivx6hAHqx0/completed
/p/Bivt6kRgJ9_/completed


/p/Bii2iekjj29/completed
/p/BihSvorlGN6/completed
/p/BihRNslgyjn/completed
/p/BihPOUYA7U8/completed
/p/BihOwbwHl1O/completed
/p/B01u-Mzget7/completed
/p/B5ECnnqHRq2/completed
/p/B5G2_yNnMq5/completed
/p/B5EJ4rbF3mY/completed
/p/Bxe-9RHAOZf/completed
/p/B-vYNRyH2F2/completed
/p/B5EMNQSF9H1/completed
/p/BtFc9zHAd2J/completed
/p/BwT7YIbHqMZ/completed
/p/BiogjpDFpW7/completed
/p/BiodXBohQrG/completed
/p/BioUPZDHWhH/completed
/p/BioS29cHn9R/completed
/p/BioRGnqF2zx/completed
/p/BioLq2AgPGY/completed
/p/BioLWS7F06U/completed
/p/BioH3BSnHCC/completed
/p/Bin1DYcg7oS/completed
/p/Binr4p4HJnU/completed
/p/BinqLKIHEUB/completed
/p/BinlNLzDawB/completed
/p/BimcxqbBOhc/completed
/p/BimJavOASf1/completed
/p/Bil5OwGHqV7/completed
/p/Bilx7sgn10U/completed
/p/Bilt3zunTlJ/completed
/p/Bilq5Heh7UG/completed
/p/BilLsz0nhj6/completed
/p/Bij2-3mgDlr/completed
/p/Bij2990ggfX/completed
/p/Bijr7Gtlosf/completed
/p/BijaQ10gw_5/completed
/p/BijJ1iNHFiL/completed
/p/Bii4MO7H3XT/completed
/p/Bii2iekjj29/completed


'\nMysql 실행\n\n## 고유 id auto-increment 생성\n1. alter table [table_name] add column id int(9) not null auto_increment primary key first;\n\n## 중복컬럼 존재하는 \n## hashtag table : hashtag, link / insta tag table : title, link 중복컬럼 그룹화를 통해 중복값 1개 남기고 제거 \n2. delete from [table_name]\n   where id\n   not in (select * from (select MIN(id) from [table_name] group by [column1, column2]) as temptable);\n\n'